In [1]:
import gurobipy as grb
import numpy as np
import itertools
import sys
import copy
import collections
import functools
import pickle

In [2]:
def iterer(*args):
    return itertools.product(*[x_ if isinstance(x_,collections.Iterable) else range(x_) for x_ in args])

In [3]:
possible_plans = [
        ['orange','black','green'],
        ['blue','black','yellow'],
        ['blue','green','orange'],
        ['yellow','green','black'],
        ['black','yellow','orange'],
        ['green','yellow','blue'],
        ['blue','orange','black'],
        ['green','orange','yellow'],
        ['black','blue','green'],
        ['orange','blue','yellow']
    ]

colormap = {
    'black'  : 0,
    'green'  : 1,
    'blue'   : 2,
    'orange' : 3,
    'yellow' : 4
}

In [4]:
"""
    places in heaps: (1 is closer to orange side, 0 to central tracking device)
       | 1|
    | 0| 4| 2|
       | 3|

    colors of cubes: (same as for orange side)
    
    black  -- 0
    green  -- 1
    blue   -- 2
    orange -- 3
    yellow -- 4
    
    numbers of heaps - 0,1,2,3,4,5 CCW starting from closest heap to central tracking device on the orange side
    
    man = 0, 1, 2 (picking 1 4 3 if from 0 for orange!)
    
    cubes:
    -1 - none
    
""";


In [5]:
class STNode():
    def __init__(self, state, upper, ref = None):
        self.state = state
        self.upper = upper
        self.nexts = []
        self.id = ref
        
    def __str__(self):
        return str(self.state)
    
    def gen_reversed_path(self):
        sn = self
        while sn != None:
            yield sn
            sn = sn.upper
    
    def get_path(self):
        return reversed(list(self.gen_reversed_path()))

In [6]:
orange_heap = np.array([
                        [-1, 0,-1],
                        [ 1, 4, 3],
                        [-1, 2,-1]
                    ],dtype=np.float)
green_heap  = np.array([
                        [-1, 0,-1],
                        [ 3, 4, 1],
                        [-1, 2,-1]
                 ],dtype=np.float)
orange_heap[orange_heap == -1] = np.nan
green_heap[green_heap == -1] = np.nan
def get_heap(heap_num, side):
    heap = None
    if heap_num in [0,1,2]:
        heap = orange_heap
    if heap_num in [3,4,5]:
        heap = green_heap
    return np.rot90(heap,side)

In [396]:
class CubesState():
    initial_cubes = np.ones((3,5),dtype=np.int)*np.nan
    magic_color = 7
    def __init__(self, heap_num, cubes = None, **kvargs):
        self.heap_num = heap_num
        self.heap = get_heap(heap_num,0)
        self.cubes = cubes if cubes else copy.deepcopy(self.initial_cubes)
        self.vars = kvargs
        self.n_picks = 0
        self.time = 0
        self.x = np.zeros((2,),dtype=np.int)
        self.a = 0
        self.params = kvargs
        self.set_defaults()
        self.last_pick = None
    
    def set_defaults(self):
        pass
    
    def remove_color(self, color):
        self.heap[self.heap == color] = np.nan
        
    def cubes_left(self):
        return np.sum(self.heap == self.heap)
    
    def get_cubes_coords(self):
        for i,j in iterer(3,3):
            if self.heap[i][j] == self.heap[i][j]:
                yield i,j
    
    def pick_cube_by(self, color, man, remove=True):
        if remove:
            self.remove_color(color)
        self.cubes[man] = np.roll(self.cubes[man],1)
        self.cubes[man][0] = int(color)
    
    def has_free_space(self,man):
        return not np.all(self.cubes[man] == self.cubes[man])
    
    def pick_cubes(self,colors,mans):
        for c,m in zip(colors, mans):
            self.pick_cube_by(c,m)
        self.n_picks += 1
        self.last_pick = mans
    
    def test_pick_cubes(self, colors, mans):
        for c, m in zip(colors,mans):
            if not self.has_free_space(m) or c != c:
                return False
        return True
    
    def is_full(self):
        return np.all(self.cubes == self.cubes)
    
    @staticmethod
    def equal_to_plan( subs, plan):
        tp = copy.deepcopy(subs)
        tp2 = np.array(list(reversed(subs)))
        for i, c in enumerate(tp):
            if c == CubesState.magic_color:
                tp[i] = plan[i]
        for i, c in enumerate(tp2):
            if c == CubesState.magic_color:
                tp2[i] = plan[i]
        return np.all(tp==plan) or np.all(tp2==plan)
    
    @staticmethod
    def count_plans_cubes(cubes,plan):
        rev_plan = np.array(list(reversed(plan)))
        has_plan = np.zeros((3,), dtype=np.bool)
        for man, s in iterer(3,3):
            has_plan[man] = has_plan[man] or CubesState.equal_to_plan(cubes[man][s:s+3],plan)
        return np.sum(has_plan)
    
    def count_plans(self, plan):
        rev_plan = np.array(list(reversed(plan)))
        has_plan = np.zeros((3,), dtype=np.bool)
        for man, s in iterer(3,3):
            has_plan[man] = has_plan[man] or CubesState.equal_to_plan(self.cubes[man][s:s+3],plan)
        return np.sum(has_plan)
    
    @staticmethod
    def sum_cubes(c1,c2):
        h1 = np.sum(c1 == c1, axis=1)
        h2 = np.sum(c2 == c2, axis=1)
        if np.any(h1 + h2 > 5):
            return None
        c = c1 + 0
        for m in range(3):
            c[m][h1[m]:h2[m]+h1[m]] = c2[m][:h2[m]]
        return c
    
    def sum_with_other_state(self, s):
        ns = copy.deepcopy(self)
        for man in range(3):
            for c in reversed(s.cubes[man]):
                if c == c and self.has_free_space(man):
                    ns.pick_cube_by(c,man,False)
        ns.time = s.time + self.time
        return ns
    
    def __str__(self):
        s = "---STATE---" + '\n'
        for l in self.heap:
            s += str(l) + '\n'
        s += "x: %d\ty: %d\ta: %d\n"%(self.x[0], self.x[1], self.a)
        #for k,v in self.vars.items():
        #    s += "%s : %s\t"%(str(k),str(v))
        for m in self.cubes:
            for c in m:
                if c == c:
                    s += "%d "%c
            s += '\n'
        s += 'time %f\tpicks %d'%(self.time, self.n_picks)
        s += '-----------'
        return s
    
    def next_picks(self):
        pass
    
    def next_moves(self):
        pass

In [397]:
class MainRobotState(CubesState):
    defaults = {
            'move_time'  :0.7,
            'pick_time'  :2+3,
            'rotate_time':1.6
        }
    banned_x = {0 : {},
                1 : {(2,0,1),(2,0,3),(1,0,1),(1,0,3)}
    }
    shift = np.array([[-1,0],[0,-1],[1,0],[0,1]])
    mans_shift = np.array([-1,0,1])
    manipulator_masks = [list(x) for x in itertools.product([0,1], repeat=3)][1:]
    def find_a_by_shift(self, sh):
        #print(self.shift, sh)
        return np.where((self.shift == sh).all(axis=1))[0][0]
    
    def set_defaults(self):
        for k,v in self.defaults.items():
            if k not in self.params:
                self.params[k] = v
    
    def new_state_x(self, x, a):
        if self.allowed_position(x,a):
            s = copy.deepcopy(self)
            s.a = a
            s.x = x
            s.time += self.get_time_between(s)
            yield s
    
    def new_state_p(self, mans):
        s = copy.deepcopy(self)
        s.time += self.params['pick_time']
        ms = [i for i, m in enumerate(mans) if m]
        x = self.get_mans_x()
        colors = [self.heap[tuple(x[m])] for m in ms]
        s.pick_cubes(colors, ms)
        return s
    
    def get_mans_x(self):
        x = np.zeros((3,2),dtype=np.int)
        x += self.x + 1
        a_index = (self.mans_shift + self.a) % 4
        # print(a_index)
        return x + self.shift[a_index]
    
    def check_mans_x(self):
        X = self.get_mans_x()
        R =[np.all(np.logical_and(x >= 0, x < 3)) and self.heap[tuple(x)] == self.heap[tuple(x)] and self.has_free_space(i) for i,x in enumerate(X)]
        return np.array(R)
        
    def get_time_between(self, s):
        return np.max(np.abs(self.x - s.x))*self.params['move_time'] + \
                np.min([np.abs(self.a - s.a), np.abs(self.a - s.a - 4), np.abs(self.a - s.a + 4)])*self.params['rotate_time']
    
    def allowed_position(self,x,a):
        return not (int(x[0]),int(x[1]),a) in self.banned_x[self.heap_num]
    
    def next_moves(self):
        cubes_left = self.cubes_left()
        if cubes_left in [3,4,5]:
            for a in range(4):
                yield from self.new_state_x(self.x, a)
        elif cubes_left == 2:
            x = np.array(list([i,j] for i,j in self.get_cubes_coords()))
            rest_cube = x.sum(axis=0) - 2
            a0 = self.find_a_by_shift(rest_cube)
            for da in range(-1,2):
                a = (a0+da)%4
                yield from self.new_state_x(self.x, a)
            for da in range(-1,2):
                a = (2-a0+da)%4
                yield from self.new_state_x(rest_cube, a)
        elif cubes_left == 1:
            last_cube = list(self.get_cubes_coords())[0]
            x = np.array([last_cube[0],last_cube[1]])
            for a,da in iterer(3,range(-1,2)):
                yield from self.new_state_x(x-1-self.shift[(a+da)%4], a)
                
    def next_picks(self):
        mans_can_pick = self.check_mans_x()
        for mans in self.manipulator_masks:
            if np.all(np.logical_or(mans_can_pick, np.logical_not(mans))): 
                yield self.new_state_p(mans)
                
    def new_states(self):
        for s in self.next_moves():
            yield from s.next_picks()

In [398]:
class PickQueryOptimizer():
    defaults = {
        'plan': [0,1,2],
        'heaps': [0,1,2],
        
    }
    def __init__(self, StateClass, heap_reducer, initial_cubes = None, **kvargs):
        self.StateClass = StateClass
        self.initial_cubes = initial_cubes
        self.params = {}
        for k,v in self.defaults.items():
            self.params[k] = copy.copy(v)
            
        for k,v in kvargs.items():
            if k in self.params.keys():
                self.params[k] = copy.copy(v)
                
        self.heap_trees_raw = {}
        self.heap_finals_safe = {} # [heap number (reduced)][plan index]
        self.heap_finals_unsafe = {} # [heap number (reduced)][plan index][num of cubes]
        self.best_tracks = {}
        self.heap_reducer = heap_reducer
                
    def set_plan(self, plan):
        if isinstance(plan[0], str):
            self.params['plan'] = [colormap[c] for c in plan]
        else:
            self.params['plan'] = plan
        
    def get_plan(self, num = None):
        if num == None:
            return self.params['plan']
        elif num < 3:
            return self.params['plan'][num]
        return self.params['plan']
    
    def get_plan_index(self):
        np_plan = np.array(self.get_plan())
        for i, p in enumerate(possible_plans):
            if np.all(np_plan == np.array([colormap[c] for c in p])):
                return i
        return None
    
    def checker_node_for_plans(self, x):
        def f(sn):
            return sn.state.count_plans(self.get_plan()) == x
        return f
    
    def reduce_by_colors(self, final_states):
        extra_colors = set([0,1,2,3,4]) - set(self.get_plan())
        cubes = {}
        for sn in final_states:
            s = sn.state
            tokey = copy.deepcopy(s.cubes)
            for ec in extra_colors:
                tokey[tokey == ec] = 6
            key = str(tokey)
            if key in cubes:
                if cubes[key].state.time > s.time:
                    cubes[key] = sn
            else:
                cubes[key] = sn
        return list(cubes.values())
    
    def build_solution_tree(self, heap_num, use_initial_cubes=True):
        initial_state = self.StateClass(heap_num, self.initial_cubes if use_initial_cubes else None)
        root = STNode(initial_state, None)
        this_wave = [root]
        next_wave = []
        i = 0
        while len(this_wave) > 0:
            for node in this_wave:
                node.nexts = [STNode(x,node, i + di) for di, x in enumerate(node.state.new_states())]
                if len(node.nexts):
                    i = node.nexts[-1].id + 1
                next_wave.extend(node.nexts)
            this_wave = next_wave
            next_wave = []
        return root
    
    def gen_final_states(self, ht_root, num_of_cubes = [5]):
        this_wave = [ht_root]
        next_wave = []
        while len(this_wave) > 0:
            for node in this_wave:
                if 5 - node.state.cubes_left() in num_of_cubes:
                    yield node
                next_wave.extend(node.nexts)
            this_wave = next_wave
            next_wave = []
        
    
    def prepare_all_heap_trees(self, status_prints=True):
        current_plan = self.get_plan()
        
        for hn in set(self.heap_reducer[hn] for hn in range(6)):
            if status_prints:
                print("Working on %d heap"%(hn))
            self.heap_trees_raw[hn] = self.build_solution_tree(hn,False)
            ht = self.heap_trees_raw[hn]
            self.heap_finals_unsafe[hn] = {}
            self.heap_finals_safe[hn] = {}
            final_states = list(self.gen_final_states(ht))
            print("final states : %d"%(len(final_states)))
            for i, p in enumerate(possible_plans):
                if status_prints:
                    print("\tplan %d"%(i))
                self.set_plan(p)
                self.heap_finals_unsafe[hn][i] = {}
                for n_of_cubes in [3,4,5]:
                    self.heap_finals_unsafe[hn][i][n_of_cubes] = self.reduce_by_colors(self.gen_final_states(ht,[n_of_cubes]))
                self.heap_finals_safe[hn][i] = list(filter(self.checker_node_for_plans(1),self.heap_finals_unsafe[hn][i][5]))
        
        self.set_plan(current_plan)
    
    def count_plans_with_magic(self, *cubes):
        cubes_sum = cubes[0]
        for cs in cubes[1:]:
            cubes_sum = CubesState.sum_cubes(cs, cubes_sum)
        return 0 if cubes_sum is None else CubesState.count_plans_cubes(cubes_sum, self.get_plan())
    
    
    
            
                
    
    def merge_heap_states(self, fs, n = 2, exclude=None):
        cubes = {}
        if not n in [2,3]:
            return None
        
        for f in iterer(*fs):
            s = None
            cubes_sum = None
            if n == 2:
                s = f[0].state, f[1].state
                cubes_sum = MainRobotState.sum_cubes(s[1].cubes,s[0].cubes)
            elif n == 3:
                s = f[0][0].state, f[0][1].state, f[1].state
                cubes_sum = MainRobotState.sum_cubes(s[2].cubes, f[0][2])
            if cubes_sum is None:
                continue
            if exclude and exclude(cubes_sum):
                continue
            key = str(cubes_sum)
            if key in cubes:
                all_nodes = None
                try:
                    if n == 2:
                        all_nodes = cubes[key][:2]
                    else:
                        all_nodes = cubes[key]
                except TypeError:
                    print(cubes[key][0], cubes[key][1])
                if sum(sn.state.time for sn in all_nodes) > sum(st.time for st in s):
                    cubes[key] = (f[0][0],f[0][1],f[1]) if n == 3 else (f[0],f[1],cubes_sum)
            else:
                cubes[key] = (f[0][0],f[0][1],f[1]) if n == 3 else (f[0],f[1],cubes_sum)
                
        return list(cubes.values())
    
        
        
    def choose_best_time(self, variants):
        mint,minv = sum( w.state.time for w in variants[0]), variants[0]
        for v in variants:
            if sum( w.state.time for w in v) > mint:
                mint = sum( w.state.time for w in v)
                minv = v
        print("min time %d"%(mint))
        return minv
    
    
    def get_best_track(self, hn_way, magic_mask = np.array([7,np.nan,7]), safe=True):
        return self.get_best_track_by_variants(self.get_variants(hn_way,magic_mask,safe), magic_mask)
    
    def get_variants(self, hn_way, magic_mask = np.array([7,np.nan,7]), safe=True):
        variants = [None,None,None]
        num_mc = np.sum(magic_mask == magic_mask)
        if safe:
            variants[:2] = [self.heap_finals_safe[hn][self.get_plan_index()] for hn in hn_way[:2]]
        else:
            variants[:2] = [self.heap_finals_unsafe[hn][self.get_plan_index()][5] for hn in hn_way[:2]]
        if num_mc > 0 or not safe:
            variants[2] = self.heap_finals_unsafe[hn_way[-1]][self.get_plan_index()][5-num_mc]
        else:
            variants[2] = self.heap_finals_safe[hn_way[-1]][self.get_plan_index()]
        return variants
    
    def get_best_track_by_variants(self, variants, magic_mask = np.array([7, np.nan, 7])):
        num_mc = np.sum(magic_mask == magic_mask)
        # step 0:
        remained_indexes0 = [(i, g.state.cubes, g.state.time) for i,g in enumerate(variants[0])]
        if num_mc > 0:
            remained_indexes0 = self.merge_heap_states_numpy([remained_indexes0,[(0,self.convert_mask_to_mans(magic_mask),0)]],1)
        finals1 = [(i, g.state.cubes, g.state.time) for i,g in enumerate(variants[1])]
        final_nodes2 = self.merge_heap_states_numpy([finals1, remained_indexes0],2)
        
        
        finals2 = [(i, g.state.cubes, g.state.time) for i,g in enumerate(variants[2])]
        
        final_nodes3 = self.merge_heap_states_numpy([final_nodes2,finals2],3)
        best = min(final_nodes3, key=lambda x : x[-1])
        ind = best[0][1][0][1], best[0][1][1],best[0][0]
        print(self.get_plan(), best[-1])
        return [variants[i][ind[i]].get_path() for i in range(3)]
        
    def prepare_best_tracks(self, magic_mask=np.array([7,np.nan,7]),safe=True):
        def lkey(l):
            return "%d%d%d"%tuple(l)
        def unlkey(s):
            return list( int(c) for c in s)
        hn_ways = [unlkey(hn_way) for hn_way in set(lkey( self.heap_reducer[hn] for hn in hn_way) for hn_way in itertools.permutations(range(6),3))]
        
        for p in possible_plans:
            print(p)
            self.set_plan(p)
            self.best_tracks[self.get_plan_index()] = {}
            for hn_way in hn_ways:
                key = lkey(hn_way)
                print('\tfor way %s'%(key))
                self.best_tracks[self.get_plan_index()][key] = self.get_best_track(hn_way, magic_mask,safe)
                
    def megamatrix_sum_numpy(self, f,s):
        megacubes0 = np.empty((len(f), 3, 5))
        megacubes1 = np.empty((len(s), 3, 5))
        for i,g in enumerate(f):
            megacubes0[i,:,:] = g
        for i,g in enumerate(s):
            megacubes1[i,:,:] = g
        megah0 = np.sum(megacubes0 == megacubes0, axis=2)[:,:,np.newaxis]
        megah1 = np.sum(megacubes1 == megacubes1, axis=2)[:,:,np.newaxis]

        megamatrixh0 = np.repeat(megah0[:,np.newaxis,:,:],len(s),axis=1)
        megamatrixh1 = np.repeat(megah1[np.newaxis,:,:,:],len(f),axis=0)
        megamatrixc1 = np.repeat(megacubes1[np.newaxis,:,:,:],len(f),axis=0)
        megamatrixc0 = np.repeat(megacubes0[:,np.newaxis,:,:],len(s),axis=1)

        temp = np.array([range(5),range(5),range(5)])
        a = np.repeat(temp[np.newaxis,:,:],len(s),axis=0)
        a = np.repeat(a[np.newaxis,:,:,:],len(f), axis=0)

        megamatrixh_2   = megamatrixh0 + megamatrixh1

        indexesc0_for_2 = np.logical_and(a >= megamatrixh0, a < megamatrixh_2)
        
        total_height_less_than_5 = np.repeat(np.all(megamatrixh_2 <= 5, axis=2), 3, axis=2)[:,:,:,np.newaxis]
        
        indexesc0_for_2 = np.logical_and(indexesc0_for_2, total_height_less_than_5)
        indexesc1_for_2 = np.logical_and(a < megamatrixh1, total_height_less_than_5)
        
        #print(indexesc0_for_2, indexesc1_for_2)
        megamatrixc0[indexesc0_for_2] = megamatrixc1[indexesc1_for_2]
        out_of_range = np.logical_and(a < 6, np.logical_not(total_height_less_than_5))
        megamatrixc0[out_of_range] = np.nan
        return megamatrixc0
    
    def convert_mask_to_mans(self, magic_mask):
        mask = np.zeros((3,5),dtype=np.float)
        mask[:] = np.nan
        print(mask[:,0].shape, magic_mask.shape)
        mask[:,0] = magic_mask
        return mask
    
    def merge_heap_states_numpy(self, fs, n = 2, safe=True):
        cubes = [[g[-2] for g in f] for f in fs]
        
        megamatrixc0 = self.megamatrix_sum_numpy(cubes[-1],cubes[-2])
        man_has_plan = np.zeros((len(fs[1]),len(fs[0]),3), dtype=np.bool)
        total_plans  = np.zeros((len(fs[1]),len(fs[0])), dtype=np.int)
        plan = np.array(self.get_plan())
        megaplan = np.repeat(plan[np.newaxis,:], len(fs[0]),axis=0)
        megaplan = np.repeat(megaplan[np.newaxis, :, :], len(fs[1]), axis=0)

        for m, shift, direction in iterer(3,3,range(-1,2,2)):
            if direction > 0:
                candidates = megamatrixc0[:,:,m,shift:shift+3:direction] 
            elif shift == 0:
                candidates = megamatrixc0[:,:,m,shift+2::direction]
            else:
                candidates = megamatrixc0[:,:,m,shift+2:shift-1:direction]
            megaplan_test = megaplan.copy()
            megaplan_test[candidates == CubesState.magic_color] = CubesState.magic_color
            man_has_plan[:,:,m] = np.logical_or(np.all(megaplan_test == candidates, axis=2), man_has_plan[:,:,m])

        total_plans = np.sum(man_has_plan,axis=2)

        same_cubes_reduce = {}

        for i,j in np.argwhere(total_plans == n):
            cubeskey = megamatrixc0[i,j,:,:]
            key = str(cubeskey)
            t = fs[1][i][-1] + fs[0][j][-1]
            if not key in same_cubes_reduce or (key in same_cubes_reduce and same_cubes_reduce[key][-1] > t):
                same_cubes_reduce[key] = ((fs[1][i][0],fs[0][j][0]),cubeskey,t)

        return list(same_cubes_reduce.values())

In [399]:
main_robot_reducer = {
    0:0,
    1:0,
    2:1,
    3:1,
    4:0,
    5:0
}


In [400]:
pqo = PickQueryOptimizer(MainRobotState, main_robot_reducer)

In [401]:
pqo.prepare_all_heap_trees()

Working on 0 heap
final states : 41670
	plan 0
	plan 1
	plan 2
	plan 3
	plan 4
	plan 5
	plan 6
	plan 7
	plan 8
	plan 9
Working on 1 heap
final states : 35249
	plan 0
	plan 1
	plan 2
	plan 3
	plan 4
	plan 5
	plan 6
	plan 7
	plan 8
	plan 9


In [402]:
saved = pqo.heap_trees_raw, pqo.heap_finals_safe, pqo.heap_finals_unsafe

In [379]:
pqo.heap_trees_raw, pqo.heap_finals_safe, pqo.heap_finals_unsafe  = saved

In [403]:
%%time
pqo.set_plan(possible_plans[0])
bt = pqo.get_best_track([0,0,1],np.array([7,np.nan,7]),False)

(3,) (3,)
[3, 0, 1] 49.39999999999999
CPU times: user 4.94 s, sys: 162 ms, total: 5.1 s
Wall time: 5.06 s


In [393]:
vs = pqo.get_variants([0,0,1], np.array([7,np.nan,7]), False)

In [390]:
len(pqo.heap_finals_unsafe[0][0][5])

1122

In [394]:
for v in vs:
    print(len(v))

1122
1122
240


In [404]:
bt

In [406]:
for bt_path in bt:
    for node in bt_path:
        print(node.state)

---STATE---
[nan  0. nan]
[1. 4. 3.]
[nan  2. nan]
x: 0	y: 0	a: 0



time 0.000000	picks 0-----------
---STATE---
[nan  0. nan]
[ 1.  4. nan]
[nan  2. nan]
x: 0	y: 0	a: 0
3 


time 5.000000	picks 1-----------
---STATE---
[nan nan nan]
[nan  4. nan]
[nan nan nan]
x: 0	y: 0	a: 1
0 3 
1 
2 
time 11.600000	picks 2-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: 0	y: 1	a: 1
0 3 
4 1 
2 
time 17.300000	picks 3-----------
---STATE---
[nan  0. nan]
[1. 4. 3.]
[nan  2. nan]
x: 0	y: 0	a: 0



time 0.000000	picks 0-----------
---STATE---
[nan  0. nan]
[ 1.  4. nan]
[nan nan nan]
x: 0	y: 0	a: 3
2 
3 

time 6.600000	picks 1-----------
---STATE---
[nan nan nan]
[nan  4. nan]
[nan nan nan]
x: 0	y: 0	a: 0
2 
0 3 
1 
time 13.200000	picks 2-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: 0	y: -1	a: 0
4 2 
0 3 
1 
time 18.900000	picks 3-----------
---STATE---
[nan  0. nan]
[1. 4. 3.]
[nan  2. nan]
x: 0	y: 0	a: 0



time 0.000000	picks 0-----------
---STATE---
[nan 

In [327]:
saved2 = saved

In [ ]:
%%time
pqo.prepare_best_tracks(np.array([[7],[np.nan],[7]]),False)

['orange', 'black', 'green']
	for way 000
2
80631
240
[0, 0, 0, 0]
4871763
min time 34
	for way 101
2
80431
240
[0, 0, 0, 0]
4857113
min time 34
	for way 011
2
80591
240
[0, 0, 0, 0]
4870257
min time 34
	for way 001
2
80631
240
[0, 0, 0, 0]
4871763
min time 34
	for way 110
2
80391
240
[0, 0, 0, 0]
4855486
min time 34
	for way 100
2
80431
240
[0, 0, 0, 0]
4857113
min time 34
	for way 010
2
80591
240
[0, 0, 0, 0]
4870257
min time 34
['blue', 'black', 'yellow']
	for way 000
2
47167
120
[0, 0, 0, 0]
1602295
min time 34
	for way 101
2
44722
120
[0, 0, 0, 0]
1520038
min time 34
	for way 011
2
44046
120
[0, 0, 0, 0]
1514517
min time 34
	for way 001
2
47167
120
[0, 0, 0, 0]
1602295
min time 34
	for way 110
2
41803
120
[0, 0, 0, 0]
1437804
min time 34
	for way 100
2
44722
120
[0, 0, 0, 0]
1520038
min time 34
	for way 010
2
44046
120
[0, 0, 0, 0]
1514517
min time 34
['blue', 'green', 'orange']
	for way 000
2
81149
240
[0, 0, 0, 0]
4891283
min time 34
	for way 101
2
80683
240
[0, 0, 0, 0]
4871643

In [142]:
saved[-1][0][0].keys()

dict_keys([5])

In [325]:
print(pqo.best_tracks[0]['000'])

None


In [252]:
pqo.fs = fs

In [254]:
fs3 = pqo.count_final_states()

7852


In [255]:
mt = 100
for (f1,f2,f3) in fs3:
    if f1.state.time + f2.state.time + f3.state.time < mt:
        mt = f1.state.time + f2.state.time + f3.state.time

In [256]:
mt

22

In [235]:
s = MainRobotState(0)
print(s)

---STATE---
[nan  0. nan]
[1. 4. 3.]
[nan  2. nan]
x: 0	y: 0	a: 0



time 0.000000	picks 0-----------


In [40]:
s.pick_cube_by(0,1)

In [41]:
s.cubes[0][0] = 1
s.cubes[0][1] = 2

In [42]:
s.cubes

array([[ 1.,  2., nan, nan, nan],
       [ 0., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [458]:
z = MainRobotState(0)

In [459]:
z.cubes[0][0] = 1
z.cubes[0][1] = 3

In [45]:
y = z.sum_with_other_state(s)

In [46]:
print(MainRobotState.sum_cubes(z.cubes, s.cubes))

[[ 1.  3.  1.  2. nan]
 [ 0. nan nan nan nan]
 [nan nan nan nan nan]]


In [47]:
y.cubes

array([[ 1.,  2.,  1.,  3., nan],
       [ 0., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [53]:
A = np.sum(y.cubes == y.cubes,axis=1)

In [57]:
B = np.array([True, False,True])

In [9]:
z = MainRobotState(0)

In [20]:
z = list(z.new_states())[-1]

In [21]:
print(z)

---STATE---
[nan nan nan]
[ 1. nan nan]
[nan nan nan]
x: 0	y: -1	a: 2
2 
3 
4 0 
time 5.000000	picks 2-----------


In [13]:
cubes1 = CubesState.sum_cubes(z.cubes, np.array([[7],[np.nan],[7]]))

In [14]:
cubes1

array([[ 2.,  7., nan, nan, nan],
       [ 3., nan, nan, nan, nan],
       [ 0.,  7., nan, nan, nan]])

In [23]:
cubes2 = z.cubes
cubes2

array([[ 2., nan, nan, nan, nan],
       [ 3., nan, nan, nan, nan],
       [ 4.,  0., nan, nan, nan]])

In [19]:
np.array([range(5),range(5),range(5)]) < np.sum(cubes1 == cubes1,axis=1)[:,np.newaxis]

array([[ True,  True, False, False, False],
       [ True, False, False, False, False],
       [ True,  True, False, False, False]])

In [27]:
temp = np.array([range(5),range(5),range(5)])
h1 = np.sum(cubes1 == cubes1, axis=1)[:,np.newaxis]
h2 = np.sum(cubes2 == cubes2, axis=1)[:,np.newaxis]
ij3 = np.logical_and(temp >= h2, temp < h2 + h1)
cubes3 = cubes2.copy()
cubes3[ij3] = cubes1[temp < h1]
cubes3

array([[ 2.,  2.,  7., nan, nan],
       [ 3.,  3., nan, nan, nan],
       [ 4.,  0.,  0.,  7., nan]])

In [43]:
a = (np.repeat(temp[np.newaxis,:],2,axis=0))

In [44]:
c = np.repeat(cubes1[np.newaxis,:],2,axis=0)

In [60]:
h = np.sum(c == c, axis=2)[:,:,np.newaxis]

In [61]:
a < h

array([[[ True,  True, False, False, False],
        [ True, False, False, False, False],
        [ True,  True, False, False, False]],

       [[ True,  True, False, False, False],
        [ True, False, False, False, False],
        [ True,  True, False, False, False]]])

In [62]:
h1 = h
h2 = h
cij = np.logical_and(a >= h1, a < h1 + h2)
c2 = c.copy()
c2[cij] = c[a < h2]


In [109]:
a = (np.repeat(temp[np.newaxis,:],2,axis=0))
np.repeat(a[np.newaxis,:,:,:],3, axis=0).shape

(3, 2, 3, 5)

In [94]:
e = np.repeat(np.all(h1 + h2 < 5,axis=1)[:,:,np.newaxis], 5,axis=2)

In [96]:
e1 = np.repeat(e, 3,axis=1)

In [99]:
c2[np.logical_and(e1, cij)]

array([2., 7., 3., 0., 7., 2., 7., 3., 0., 7.])

In [106]:
np.repeat(c2[np.newaxis,:,:,:], 2, axis=0)

array([[[[ 2.,  7.,  2.,  7., nan],
         [ 3.,  3., nan, nan, nan],
         [ 0.,  7.,  0.,  7., nan]],

        [[ 2.,  7.,  2.,  7., nan],
         [ 3.,  3., nan, nan, nan],
         [ 0.,  7.,  0.,  7., nan]]],


       [[[ 2.,  7.,  2.,  7., nan],
         [ 3.,  3., nan, nan, nan],
         [ 0.,  7.,  0.,  7., nan]],

        [[ 2.,  7.,  2.,  7., nan],
         [ 3.,  3., nan, nan, nan],
         [ 0.,  7.,  0.,  7., nan]]]])

In [129]:
for x in z.new_states():
    print(x)

---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -1	y: -1	a: 0
0 4 2 
3 
1 
time 10.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: 0	y: 0	a: 0
4 2 
0 3 
1 
time 10.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -1	y: 1	a: 0
4 2 
3 
0 1 
time 10.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: 0	y: 0	a: 1
0 4 2 
3 
1 
time 11.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -1	y: 1	a: 1
4 2 
0 3 
1 
time 11.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -2	y: 0	a: 1
4 2 
3 
0 1 
time 11.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -1	y: 1	a: 2
0 4 2 
3 
1 
time 10.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -2	y: 0	a: 2
4 2 
0 3 
1 
time 10.000000	picks 4-----------
---STATE---
[nan nan nan]
[nan nan nan]
[nan nan nan]
x: -1	y: -1

In [69]:
next(z.get_cubes_coords())

(1, 2)

In [83]:
for x in z.next_moves():
    print(x)

---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: 0	y: 0	a: 3
2 
3 
1 
time 5.000000	picks 2-----------
---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: 0	y: 0	a: 0
2 
3 
1 
time 4.000000	picks 2-----------
---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: 0	y: 0	a: 1
2 
3 
1 
time 5.000000	picks 2-----------
---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: -1	y: 0	a: 3
2 
3 
1 
time 6.000000	picks 2-----------
---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: -1	y: 0	a: 0
2 
3 
1 
time 5.000000	picks 2-----------
---STATE---
[nan  0. nan]
[nan  4. nan]
[nan nan nan]
x: -1	y: 0	a: 1
2 
3 
1 
time 6.000000	picks 2-----------


In [241]:
{[1,2]}

TypeError: unhashable type: 'list'

In [112]:
magic_mask = np.array([[7],[np.nan],[7]])
np.repeat(np.repeat(magic_mask[np.newaxis,np.newaxis,:,:],2,axis=0),3,axis=1)

array([[[[ 7.],
         [nan],
         [ 7.]],

        [[ 7.],
         [nan],
         [ 7.]],

        [[ 7.],
         [nan],
         [ 7.]]],


       [[[ 7.],
         [nan],
         [ 7.]],

        [[ 7.],
         [nan],
         [ 7.]],

        [[ 7.],
         [nan],
         [ 7.]]]])